# Image Classifier Project Development


In [2]:
import torch
from torch import nn, optim
from torchvision import datasets, transforms, models
from collections import OrderedDict
import matplotlib.pyplot as plt
import numpy as np
import json
from PIL import Image
from tqdm import tqdm

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

ModuleNotFoundError: No module named 'tqdm'

In [ ]:
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

## Step 2: Load and Preprocess Data

In [ ]:


# Define transforms for training, validation, and testing
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(30),
        transforms.ColorJitter(brightness=0.2, contrast=0.2),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

# Load datasets with ImageFolder
image_datasets = {
    'train': datasets.ImageFolder(train_dir, transform=data_transforms['train']),
    'valid': datasets.ImageFolder(valid_dir, transform=data_transforms['valid']),
    'test': datasets.ImageFolder(test_dir, transform=data_transforms['test'])
}

# Create dataloaders
batch_size = 64
dataloaders = {
    'train': torch.utils.data.DataLoader(image_datasets['train'], batch_size=batch_size, shuffle=True),
    'valid': torch.utils.data.DataLoader(image_datasets['valid'], batch_size=batch_size),
    'test': torch.utils.data.DataLoader(image_datasets['test'], batch_size=batch_size)
}

# Load category names
with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

print("Number of training images:", len(image_datasets['train']))
print("Number of validation images:", len(image_datasets['valid']))
print("Number of test images:", len(image_datasets['test']))
print("Number of categories:", len(cat_to_name))

In [ ]:
# Load category names
with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)


print("Number of training images:", len(image_datasets['train']))
print("Number of validation images:", len(image_datasets['valid']))
print("Number of test images:", len(image_datasets['test']))
print("Number of categories:", len(cat_to_name))


## Step 3: Build and Train Model

In [ ]:
# Load pretrained model (VGG16)
model = models.vgg16(pretrained=True)

# Freeze parameters
for param in model.parameters():
    param.requires_grad = False

# Define classifier
classifier = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(25088, 4096)),
    ('relu1', nn.ReLU()),
    ('dropout1', nn.Dropout(0.5)),
    ('fc2', nn.Linear(4096, 102)),
    ('output', nn.LogSoftmax(dim=1))
]))

model.classifier = classifier

# Define loss and optimizer
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

# Add learning rate scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, verbose=True)

# Move model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

## Step 4: Train the Network

In [ ]:
epochs = 5
steps = 0
print_every = 40
best_accuracy = 0
train_losses, valid_losses = [], []

for epoch in range(epochs):
    running_loss = 0
    model.train()
    
    # Training loop with progress bar
    train_bar = tqdm(dataloaders['train'], desc=f'Epoch {epoch+1}/{epochs}')
    for inputs, labels in train_bar:
        steps += 1
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        train_bar.set_postfix({'train_loss': loss.item()})
        
        if steps % print_every == 0:
            model.eval()
            valid_loss = 0
            accuracy = 0
            
            with torch.no_grad():
                for inputs, labels in dataloaders['valid']:
                    inputs, labels = inputs.to(device), labels.to(device)
                    outputs = model(inputs)
                    valid_loss += criterion(outputs, labels).item()
                    
                    ps = torch.exp(outputs)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
            
            train_loss = running_loss/print_every
            valid_loss = valid_loss/len(dataloaders['valid'])
            accuracy = accuracy/len(dataloaders['valid'])
            
            train_losses.append(train_loss)
            valid_losses.append(valid_loss)
            
            scheduler.step(valid_loss)
            
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'accuracy': accuracy,
                }, 'best_model.pth')
            
            print(f"Epoch {epoch+1}/{epochs}.. 
                  Train loss: {train_loss:.3f}.. 
                  Validation loss: {valid_loss:.3f}.. 
                  Accuracy: {accuracy:.3f}")
            
            running_loss = 0
            model.train()

print("Training completed!")
print(f"Best validation accuracy: {best_accuracy:.3f}")

## Step 5: Test the Network

In [ ]:
model.eval()
test_loss = 0
test_accuracy = 0

with torch.no_grad():
    test_bar = tqdm(dataloaders['test'], desc='Testing')
    for inputs, labels in test_bar:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        test_loss += criterion(outputs, labels).item()
        
        ps = torch.exp(outputs)
        top_p, top_class = ps.topk(1, dim=1)
        equals = top_class == labels.view(*top_class.shape)
        test_accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
        
        test_bar.set_postfix({'test_accuracy': test_accuracy/len(dataloaders['test'])})

test_loss = test_loss/len(dataloaders['test'])
test_accuracy = test_accuracy/len(dataloaders['test'])

print(f"\nTest accuracy: {test_accuracy:.3f}")
print(f"Test loss: {test_loss:.3f}")

## Step 6: Save Checkpoint

In [ ]:
checkpoint = {
    'arch': 'vgg16',
    'classifier': model.classifier,
    'state_dict': model.state_dict(),
    'class_to_idx': image_datasets['train'].class_to_idx,
    'optimizer_state': optimizer.state_dict(),
    'epochs': epochs,
    'accuracy': test_accuracy
}

torch.save(checkpoint, 'checkpoint.pth')
print("Checkpoint saved!")

## Step 7: Load Checkpoint

In [ ]:
def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    
    model = models.vgg16(pretrained=True)
    model.classifier = checkpoint['classifier']
    model.load_state_dict(checkpoint['state_dict'])
    model.class_to_idx = checkpoint['class_to_idx']
    
    return model

# Test loading checkpoint
model = load_checkpoint('checkpoint.pth')
print("Checkpoint loaded successfully!")

## Step 8: Process Image

In [ ]:
def process_image(image_path):
    ''' Process image for model input '''
    # Open image
    img = Image.open(image_path)
    
    # Define transforms
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )
    ])
    
    # Process image
    img_tensor = preprocess(img)
    
    return img_tensor

In [ ]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

## Step 9: Class Prediction

In [ ]:
def predict(image_path, model, topk=5):
    ''' Predict class from an image '''
    # Process image
    img = process_image(image_path)
    img = img.unsqueeze_(0).to(device)
    
    # Set model to evaluation mode
    model.eval()
    
    # Make prediction
    with torch.no_grad():
        output = model(img)
        ps = torch.exp(output)
        top_p, top_class = ps.topk(topk, dim=1)
    
    # Convert indices to classes
    idx_to_class = {v: k for k, v in model.class_to_idx.items()}
    classes = [idx_to_class[i] for i in top_class.cpu().numpy()[0]]
    probs = top_p.cpu().numpy()[0]
    
    return probs, classes

## Step 10: Sanity Checking

In [ ]:
def plot_prediction(image_path, model, topk=5):
    ''' Display image and prediction '''
    # Get prediction
    probs, classes = predict(image_path, model, topk)
    flower_names = [cat_to_name[c] for c in classes]
    
    # Plot image
    img = Image.open(image_path)
    fig, (ax1, ax2) = plt.subplots(figsize=(12,4), nrows=1, ncols=2)
    
    # Plot flower
    ax1.imshow(img)
    ax1.axis('off')
    ax1.set_title(f'Actual: {cat_to_name[image_path.split("/")[-2]]}')
    
    # Plot probabilities
    ax2.barh(np.arange(topk), probs)
    ax2.set_yticks(np.arange(topk))
    ax2.set_yticklabels(flower_names)
    ax2.set_xlabel('Probability')
    ax2.set_title('Top 5 Predictions')
    
    plt.tight_layout()
    plt.show()

# Test prediction
test_image = 'flowers/test/1/image_06743.jpg'
plot_prediction(test_image, model)